In [1]:
import time


def clock_v1(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result!r}')
        return result
    return clocked

In [3]:
import time
import functools


def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.perf_counter()
        result = func(*args, **kwargs)
        elapsed = time.perf_counter() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = [f'{k}={v!r}' for k, v in kwargs.items()]
            arg_lst.append(', '.join(pairs))
        arg_str = ', '.join(arg_lst)
        print(f'[{elapsed:0.8f}s] {name}({arg_str}) -> {result!r}')
        return result
    return clocked

In [4]:
@clock
def snooze(seconds):
    time.sleep(seconds)


@clock
def factorial(n):
    return 1 if n < 2 else n*factorial(n-1)


if __name__ == '__main__':
    print('*' * 40, 'Calling snooze(.123)')
    snooze(.123)
    print('*' * 40, 'Calling factorial(6)')
    print('6! =', factorial(6))

**************************************** Calling snooze(.123)
[0.12272640s] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000180s] factorial(1) -> 1
[0.00006500s] factorial(2) -> 2
[0.00010640s] factorial(3) -> 6
[0.00015500s] factorial(4) -> 24
[0.00020270s] factorial(5) -> 120
[0.00026450s] factorial(6) -> 720
6! = 720


## Stacked Decoraters and the built-in decorater Cache

In [8]:
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)


if __name__ == '__main__':
    print(fibonacci(6))

[0.00000030s] fibonacci(0) -> 0
[0.00000050s] fibonacci(1) -> 1
[0.00016930s] fibonacci(2) -> 1
[0.00000020s] fibonacci(1) -> 1
[0.00000050s] fibonacci(0) -> 0
[0.00000040s] fibonacci(1) -> 1
[0.00004930s] fibonacci(2) -> 1
[0.00006960s] fibonacci(3) -> 2
[0.00025940s] fibonacci(4) -> 3
[0.00000020s] fibonacci(1) -> 1
[0.00000020s] fibonacci(0) -> 0
[0.00000020s] fibonacci(1) -> 1
[0.00001870s] fibonacci(2) -> 1
[0.00003850s] fibonacci(3) -> 2
[0.00000020s] fibonacci(0) -> 0
[0.00000020s] fibonacci(1) -> 1
[0.00001960s] fibonacci(2) -> 1
[0.00000020s] fibonacci(1) -> 1
[0.00000040s] fibonacci(0) -> 0
[0.00000020s] fibonacci(1) -> 1
[0.00004960s] fibonacci(2) -> 1
[0.00006860s] fibonacci(3) -> 2
[0.00010690s] fibonacci(4) -> 3
[0.00016460s] fibonacci(5) -> 5
[0.00044370s] fibonacci(6) -> 8
8


In [11]:
@functools.cache
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-2) + fibonacci(n-1)


if __name__ == '__main__':
    print(fibonacci(6))

[0.00000030s] fibonacci(0) -> 0
[0.00000050s] fibonacci(1) -> 1
[0.00024370s] fibonacci(2) -> 1
[0.00000060s] fibonacci(3) -> 2
[0.00026340s] fibonacci(4) -> 3
[0.00000050s] fibonacci(5) -> 5
[0.00039690s] fibonacci(6) -> 8
8


## singledispatch

In [13]:
from functools import singledispatch
from collections import abc
import fractions
import decimal
import html
import numbers

@singledispatch
def htmlize(obj: object) -> str:
    content = html.escape(repr(obj))
    return f'<pre>{content}</pre>'

@htmlize.register
def _(text: str) -> str:
    content = html.escape(text).replace('\n', '<br>\n')
    return f'<p>{content}</p>'

@htmlize.register
def _(seq: abc.Sequence) -> str:
    inner = '</li>\n<li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

@htmlize.register
def _(n: numbers.Integral) -> str:
    return f'<pre>{n} (0x{n:x})</pre>'

@htmlize.register
def _(n: bool) -> str:
    return f'<pre>{n}</pre>'

@htmlize.register(fractions.Fraction)
def _(x) -> str:
    frac = fractions.Fraction(x)
    return f'<pre>{frac.numerator}/{frac.denominator}</pre>'

@htmlize.register(decimal.Decimal)
@htmlize.register(float)
def _(x) -> str:
    frac = fractions.Fraction(x).limit_denominator()
    return f'<pre>{x} ({frac.numerator}/{frac.denominator})</pre>'

In [14]:
htmlize({1, 2, 3})

'<pre>{1, 2, 3}</pre>'

In [15]:
htmlize(abs)

'<pre>&lt;built-in function abs&gt;</pre>'

In [16]:
htmlize('Heimlich & Co.\n- a game')

'<p>Heimlich &amp; Co.<br>\n- a game</p>'

In [17]:
htmlize(42)

'<pre>42 (0x2a)</pre>'

In [18]:
print(htmlize(['alpha', 66, {3, 2, 1}]))

<ul>
<li><p>alpha</p></li>
<li><pre>66 (0x42)</pre></li>
<li><pre>{1, 2, 3}</pre></li>
</ul>


In [19]:
htmlize(True)

'<pre>True</pre>'

In [20]:
htmlize(fractions.Fraction(2, 3))

'<pre>2/3</pre>'

In [21]:
htmlize(2/3)

'<pre>0.6666666666666666 (2/3)</pre>'

In [22]:
htmlize(decimal.Decimal('0.02380952'))

'<pre>0.02380952 (1/42)</pre>'

## Parameterized Decorators

In [ ]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):
    def decorate(func):
        def clocked(*_args):
            t0 = time.perf_counter()
            _result = func(*_args)
            elapsed = time.perf_counter() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(fmt.format(**locals()))
            return _result
        return clocked
    return decorate

if __name__ == '__main__':

    @clock()
    def snooze(seconds):
        time.sleep(seconds)

    for i in range(3):
        snooze(.123)

## Class Decorators

In [ ]:
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

class clock:

    def __init__(self, fmt=DEFAULT_FMT):
        self.fmt = fmt

    def __call__(self, func):
        def clocked(*_args):
            t0 = time.perf_counter()
            _result = func(*_args)
            elapsed = time.perf_counter() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)
            print(self.fmt.format(**locals()))
            return _result
        return clocked